### Example radon transform figures

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subjects
from neuropy import plotting
from neuropy.analyses import Pf1D
from neuropy.analyses import Decode1d

sess = subjects.nsd.ratUday2[0]

maze = sess.paradigm["maze"].flatten()
post = sess.paradigm["post"].flatten()
neurons = sess.neurons.get_neuron_type(neuron_type="pyr")
pos = sess.maze
# pos.t_start = pos.t_start +0.5
# run = sess.run
pf = Pf1D(
    neurons=neurons,
    position=pos,
    speed_thresh=4,
    sigma=4,
    grid_bin=2,
    # epochs=run,
    frate_thresh=1,
)
pf_neurons = neurons.get_by_id(pf.neuron_ids)
epochs = sess.pbe.time_slice(maze[0], maze[0]+1000)
decode = Decode1d(
    neurons=pf_neurons, ratemap=pf, epochs=epochs, bin_size=0.02
)

In [ ]:
# %matplotlib widget
from numpy.random import default_rng
from neuropy.analyses.decoders import radon_transform

n_posteriors = len(decode.posterior)
posterior_ind = default_rng().integers(0,n_posteriors,10)
# posterior_ind = [382, 42, 131, 162, 287, 284, 127, 331, 294, 205]
arrs = [decode.posterior[i] for i in posterior_ind]

_, axs = plt.subplots(2, 5)
axs = axs.reshape(-1)

for i, arr in enumerate(arrs):

    score, velocity, intercept = radon_transform(arr, dt=0.02, dx=2, neighbours=2)
    arr = np.apply_along_axis(np.convolve, axis=0, arr=arr, v=np.ones(2 * 2 + 1))
    t = np.arange(arr.shape[1]) * 0.02
    pos = np.arange(arr.shape[0]) * 2

    axs[i].pcolormesh(t, pos,arr, cmap="hot")
    axs[i].plot(t, velocity * t + intercept, color="w")
    axs[i].set_ylim([pos.min(), pos.max()])


### Decoding here

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subjects
from neuropy import plotting
from neuropy.utils import signal_process
from neuropy.analyses import Pf1D
from neuropy.analyses import Decode1d

sessions = subjects.nsd.ratUday2

In [ ]:
for sub, sess in enumerate(sessions):
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type(neuron_type="pyr")
    pos = sess.maze
    # pos.t_start = pos.t_start +0.5
    # run = sess.run
    pf = Pf1D(
        neurons=neurons,
        position=pos,
        speed_thresh=4,
        sigma=4,
        grid_bin=2,
        # epochs=run,
        frate_thresh=1,
    )
    pf_neurons = neurons.get_by_id(pf.neuron_ids)
    epochs = sess.pbe.time_slice(maze[0], maze[1])
    decode = Decode1d(
        neurons=pf_neurons, ratemap=pf, epochs=epochs, bin_size=0.02
    )


In [ ]:
from hfuncs import plot_in_bokeh
import bokeh.plotting as bplot

# bplot.output_file(subjects.figpath_sd/'test_wake_decoding.html')
bplot.output_notebook()
p = plot_in_bokeh(x=epochs.starts/3600,y=decode.score,img_arr=decode.posterior
,color_by=decode.score,palette='jet',size=10)
# p.line(pos.time,pos.x/350 + 0.2,line_width=2,color='black')
bplot.show(p)

### Plotting decoding stats if calculated via script

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subjects
from neuropy import plotting
from neuropy.utils import signal_process
from neuropy.analyses import Pf1D
from neuropy.analyses import Decode1d

sessions = subjects.nsd.ratUday2

In [ ]:
for sub,sess in enumerate(sessions):

    replay_pbe = sess.replay_pbe

In [ ]:
replay_pbe.to_dataframe()

In [ ]:
from hfuncs import plot_in_bokeh
import bokeh.plotting as bplot

# bplot.output_file(subjects.figpath_sd/'test_wake_decoding.html')
t = replay_pbe.starts
score = replay_pbe.to_dataframe()['replay_score']
img = replay_pbe.metadata['posterior']
p_val = replay_pbe.to_dataframe()['replay_p_value']
is_replay = np.where(p_val<0.10,p_val,0)
bplot.output_notebook()
p = plot_in_bokeh(x=replay_pbe.starts/3600 - sess.paradigm['post'].flatten()[0]/3600,y=score,img_arr=img,color_by=p_val,palette='jet_r',size=10)
# p.line(pos.time,pos.x/350 + 0.2,line_width=2,color='black')
bplot.show(p)

### Decoding theta sequences

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subjects
from neuropy import plotting
from neuropy.utils import signal_process
from neuropy.analyses import Pf1D
from neuropy.analyses import Decode1d

sessions = subjects.nsd.ratUday2

In [ ]:
for sub, sess in enumerate(sessions):
    maze = sess.paradigm["maze"]
    post = sess.paradigm["post"]
    neurons = sess.neurons.get_neuron_type(neuron_type="pyr")
    pos = sess.lin_maze
    # pos.t_start = pos.t_start +0.5
    run = sess.run
    pf = Pf1D(
        neurons=neurons,
        position=pos,
        speed_thresh=4,
        sigma=4,
        grid_bin=2,
        # epochs=run,
        frate_thresh=1,
    )
    pf_neurons = neurons.get_by_id(pf.ratemap.neuron_ids)
    epochs = sess.pbe.time_slice(maze[0], maze[1])
    decode = Decode1d(
        neurons=pf_neurons, ratemap=pf.ratemap, epochs=epochs, bin_size=0.02,slideby=0.01
    )


In [ ]:
from hfuncs import plot_in_bokeh
import bokeh.plotting as bplot

# bplot.output_file(subjects.figpath_sd/'test_wake_decoding.html')
bplot.output_notebook()
p = plot_in_bokeh(x=epochs.starts/3600 - sess.paradigm['maze'][0]/3600,y=decode.score,img_arr=decode.spkcount,color_by=decode.score,palette='jet_r',size=10)
# p.line(pos.time,pos.x/350 + 0.2,line_width=2,color='black')
bplot.show(p)